In [1]:
import requests
import pandas as pd

In [2]:
df = pd.read_csv("Online Retail.csv")

In [3]:
df.head()

df = df.dropna()

# for some reason CustomerID is interpreted as float.  Change to int
df['CustomerID'] = df['CustomerID'].astype(int)

In [4]:
r = requests.get("http://elasticsearch:9200")
if r.status_code != 200:
    
    print("Error talking to Elasticsearch")
else:
    print(r.json())

{'name': 'GJfcAQP', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'IhzXzag6T0qNX9bzo2wwUw', 'version': {'number': '5.6.16', 'build_hash': '3a740d1', 'build_date': '2019-03-13T15:33:36.565Z', 'build_snapshot': False, 'lucene_version': '6.6.1'}, 'tagline': 'You Know, for Search'}


In [5]:
def print_es_indices():
    r = requests.get("http://elasticsearch:9200/_cat/indices?v")
    if r.status_code != 200:
        print("Error listing indices")
    else:
        print(r.text)
        
print_es_indices()

health status index     uuid                   pri rep docs.count docs.deleted store.size pri.store.size
yellow open   .kibana   we4aWTmXTv6gQUN3zBGFKg   1   1          1            0      3.2kb          3.2kb
yellow open   recsystem q7RIh614SWCsiMISiUspdg   5   1          0            0       810b           810b



In [6]:
def delete_es_index(index):
    r = requests.delete("http://elasticsearch:9200/{}".format(index))
    if r.status_code != 200:
        print("Error deleting index")
    else:
        print("Index deleted")

In [7]:
delete_es_index("recsystem")
delete_es_index(".kibana")

Index deleted
Index deleted


In [8]:
print_es_indices()

health status index uuid pri rep docs.count docs.deleted store.size pri.store.size



In [9]:
def create_index(index, index_config):
    r = requests.put("http://elasticsearch:9200/{}".format(index),
                     json=index_config)
    if r.status_code != 200:
        print(r.status_code,":Error creating index")
    else:
        print("Created")

In [10]:
index_config = {
    "mappings": {
        "customer_docs": {
            "properties": {
                "InvoiceNo": {"type": "integer"},
                "CustomerID": {"type": "integer"},
                "InvoiceDate":{"type":"date"},
                "Country":{"type":"string"},
                "stockcodes":{"type": "string"},
                "descriptions":{"type": "string"},
                "quantities":{"type": "integer"},
                "unitprices":{"type": "float"}
            }
        }
    }
}

In [11]:
create_index("recsystem", index_config)

Created


In [12]:
def fling_message(index, doctype, msg):
    r = requests.post("http://elasticsearch:9200/{}/{}".format(index, doctype),
                      json=msg)
    if r.status_code != 201:
        print(r.status_code,":Error sending message")
    else:
        print("message sent")

In [13]:
# group by invoice num
invoice_groups = df.groupby('InvoiceNo')

# iterate over each group (each invoice)
for invoice_name, invoice in invoice_groups:
    basket = {}
    stockcodes = []
    descriptions = []
    quantities = []
    unitprices = []
    # iterate over rows in this invoice dataframe
    for row_index, row in invoice.iterrows():
        # these fields are the same for each row, so doesn't matter if we keep overwriting
        basket['InvoiceNo'] = row['InvoiceNo']
        basket['CustomerID'] = row['CustomerID']
        basket['InvoiceDate'] = row['InvoiceDate']
        basket['Country'] = row['Country']
        # these fields are different for each row, so we append to lists
        stockcodes.append(row['StockCode'])
        descriptions.append(row['Description'])
        quantities.append(row['Quantity'])
        unitprices.append(row['UnitPrice'])
    basket['StockCodes'] = stockcodes
    basket['Descriptions'] = descriptions
    basket['Quantities'] = quantities
    basket['UnitPrices'] = unitprices

    # now you should fling to elasticsearch!
    #print(basket)
    fling_message("recsystem","customer_docs",basket)

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

...

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
400 :Error sending message
4

In [14]:
def execute_es_query(index, query):
    r = requests.get("http://elasticsearch:9200/{}/_search".format(index),
                     json=query)
    if r.status_code != 200:
        print("Error executing query")
        return None
    else:
        return r.json()

In [ ]:
query = {
    "size": 0,
    "query": {"match_all": {} 
    },
    "aggs": {
        "highly_correlated_words": {
            "significant_terms": {
                "field": "stockcodes",
                "min_doc_count": 1,
            }
        }
    }
}


res = execute_es_query("recsystem", query)